# Data Manipulation

The notebook shows how to join two tables on Trino to create a new table. Such a table can then be used for further analysis or for creating visualizations in Apache Superset.

In [1]:
import os
import json
import time
import pathlib
from dotenv import load_dotenv
import trino
import pandas as pd

### Injecting Credentials

In order to run this notebook, we need credentials to connect with the Trino server to retrieve and create tables.

In an automated environment, the credentials can be specified in a pipeline's environment variables or through Openshift secrets. 

For running the notebook in automation in an elyra pipeline, the environment variables can be updated in the notebook "Properties" in the pipeline UI or under `"env_vars"` in the `demo1.pipeline yaml` file.

For running the notebook in a local environment, we will define them as environment variables in a `credentials.env` file, and load them using dotenv. An example of what the contents of `credentials.env` could look like is shown below

```
# trino credentials
TRINO_USER=xxx
TRINO_PASSWD=xxx
TRINO_HOST=trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org
TRINO_PORT=443
```

In [2]:
# file to store runtime kfpipeline metrics
metrics_file_path = './mlpipeline-metrics.json'

In [3]:
# Load credentials
dotenv_dir = "/opt/app-root/src/aicoe-osc-demo"
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

# Join two tables on Trino

In [4]:
# use trino password env-var to hold token values
JWT_TOKEN = os.environ['TRINO_PASSWD']
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=os.environ['TRINO_PORT'],
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(JWT_TOKEN),
)
cur = conn.cursor()

In [5]:
# Generate column names for df_emissions table
# and remove isin column to avoid duplication
# of key column in the join operation
cur.execute("show columns from osc_datacommons_dev.urgentem.itr_emissions_1")
res = cur.fetchall()
df_columns = [i[0] for i in res]
df_columns.remove("isin")
no_of_cols_in_table1= len(df_columns)
df_columns = ["osc_datacommons_dev.urgentem.itr_emissions_1." + i for i in df_columns]
df_columns = ", ".join(df_columns)

In [6]:
# Write the join_query
join_query = f"CREATE TABLE if not exists osc_datacommons_dev.urgentem.itr_emissions_joined AS\
              SELECT osc_datacommons_dev.urgentem.itr_emissions_2.*, {df_columns} \
              FROM osc_datacommons_dev.urgentem.itr_emissions_2 \
              LEFT JOIN osc_datacommons_dev.urgentem.itr_emissions_1 \
              ON osc_datacommons_dev.urgentem.itr_emissions_1.isin=osc_datacommons_dev.urgentem.itr_emissions_2.isin"
join_query

'CREATE TABLE if not exists osc_datacommons_dev.urgentem.itr_emissions_joined AS              SELECT osc_datacommons_dev.urgentem.itr_emissions_2.*, osc_datacommons_dev.urgentem.itr_emissions_1.company_name, osc_datacommons_dev.urgentem.itr_emissions_1.target_type, osc_datacommons_dev.urgentem.itr_emissions_1.scope, osc_datacommons_dev.urgentem.itr_emissions_1.coverage_s1, osc_datacommons_dev.urgentem.itr_emissions_1.coverage_s2, osc_datacommons_dev.urgentem.itr_emissions_1.coverage_s3, osc_datacommons_dev.urgentem.itr_emissions_1.reduction_ambition, osc_datacommons_dev.urgentem.itr_emissions_1.base_year, osc_datacommons_dev.urgentem.itr_emissions_1.end_year, osc_datacommons_dev.urgentem.itr_emissions_1.start_year, osc_datacommons_dev.urgentem.itr_emissions_1.base_year_ghg_emissions_s1_tco2e, osc_datacommons_dev.urgentem.itr_emissions_1.base_year_ghg_emissions_s1s2_tco2e, osc_datacommons_dev.urgentem.itr_emissions_1.base_year_ghg_emissions_s3_tco2e, osc_datacommons_dev.urgentem.itr_emi

In [7]:
# Execute the join query
t = time.time()
cur.execute(join_query)
time_to_join_tables = time.time() - t
cur.fetchall()

[[0]]

In [8]:
# Check if the joined table is there
cur.execute("select * from osc_datacommons_dev.urgentem.itr_emissions_joined LIMIT 5")
pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,BANCO SANTANDER,ES0113900J37,5705946,SAN SM,1,2,Spain,Europe,693.1,4.1,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN
1,ENEL SPA,IT0003128367,7144569,ENEL IM,1,4,Italy,Europe,1557.4,1116.2,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN
2,GAZPROM,RU0007661625,B59L4L7,GAZP RM,2,0,Russian Federation,Europe,6033.4,2003.8,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN
3,BARCLAYS PLC,GB0031348658,3134865,BARC LN,1,3,United Kingdom,Europe,709.2,9.7,...,1.0,None,0.37,2018.0,2025.0,2018.0,None,"282,593","1,412,965",0.0
4,BAYER AG-REG,DE000BAY0017,5069211,BAYN GR,1,8,Germany,Europe,385.5,69.2,...,NaN,None,NaN,NaN,NaN,NaN,None,None,None,NaN


In [9]:
metrics = {
    'metrics': [
        {
            'name': 'time_to_join_tables',
            'numberValue':  time_to_join_tables,
            'format': "RAW",
        },
        {
            'name': 'no_of_cols_in_table1',
            'numberValue':  no_of_cols_in_table1,
            'format': "RAW",
        }
    ]}

In [10]:
pathlib.Path(metrics_file_path).parent.mkdir(parents=True, exist_ok=True)
pathlib.Path(metrics_file_path).write_text(json.dumps(metrics))

170

# Conclusion

In this notebook, we showed how to read from existing Trino tables and perform a basic join operation on them to create a new table. The table can now be used in a Superset dashboard for visualization. 